# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_excercise').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1524112428918_0004,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
from pyspark.ml.clustering import KMeans

In [4]:
df = spark.read.csv("hdfs://...hack_data.csv",header=True,inferSchema=True)

In [5]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [6]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)

In [7]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

### Turn data into "feature" format for Spark ML. No "label" column because this is usupervised learning

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
# copy, paste, and edit print out for next cell

df.columns

['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'Location', 'WPM_Typing_Speed']

In [10]:
assembler = VectorAssembler(inputCols = ['Session_Connection_Time',
                                         'Bytes Transferred', 
                                         'Kali_Trace_Used', 
                                         'Servers_Corrupted', 
                                         'Pages_Corrupted',
                                         'WPM_Typing_Speed'],
                           outputCol = "features")

# if you wanted to take all the features without writing it all out
# assembler = VectorAssembler(inputCols = df.columns, outputCol = "features")

In [18]:
feature_data = assembler.transform(df)

### Scale Data

In [15]:
from pyspark.ml.feature import StandardScaler

In [17]:
scaler = StandardScaler(inputCol = "features", outputCol = "scaledFeatures",
                            withStd = True, withMean = False)

In [20]:
ml_data = scaler.fit(feature_data).transform(feature_data)

In [27]:
ml_data.show(5, truncate = False)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+------------------------------------------------------------------------------------------------------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location              |WPM_Typing_Speed|features                        |scaledFeatures                                                                                                    |
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+------------------------------------------------------------------------------------------------------------------+
|8.0                    |391.09           |1              |2.96             |7.0            |Slovenia              |72.37           |[8.0,391.09,1.0,2.96,

### Train Model

In [36]:
# make a cluster for 2 and 3 hackers.

kmeans2 = KMeans(featuresCol = "scaledFeatures", k = 2)
kmeans3 = KMeans(featuresCol = "scaledFeatures", k = 3)

In [54]:
# You are not using a train and test set.
# you are making predictions on the same set

hacker_cluster_2 = kmeans2.fit(ml_data).transform(ml_data)
hacker_cluster_3 = kmeans3.fit(ml_data).transform(ml_data)

### Evaluate 

Each cluster represents a particular hackers work because those hacking jobs are similar to each other.

You just have to count how many hacks are in each cluster. The answer, correct number of clusters/hackers, will have an even amount of hacks in each cluster.

In [57]:
hacker_cluster_2.select("features","prediction").show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[8.0,391.09,1.0,2...|         1|
|[20.0,720.99,0.0,...|         1|
|[31.0,356.32,1.0,...|         1|
|[2.0,228.08,1.0,2...|         1|
|[20.0,408.5,0.0,3...|         1|
|[1.0,390.69,1.0,2...|         1|
|[18.0,342.97,1.0,...|         1|
|[22.0,101.61,1.0,...|         1|
|[15.0,275.53,1.0,...|         1|
|[12.0,424.83,1.0,...|         1|
|[15.0,249.09,1.0,...|         1|
|[32.0,242.48,0.0,...|         1|
|[23.0,514.54,0.0,...|         1|
|[9.0,284.77,0.0,3...|         1|
|[27.0,779.25,1.0,...|         1|
|[12.0,307.31,1.0,...|         1|
|[21.0,355.94,1.0,...|         1|
|[10.0,372.65,0.0,...|         1|
|[20.0,347.23,1.0,...|         1|
|[22.0,456.57,0.0,...|         1|
+--------------------+----------+
only showing top 20 rows

In [51]:
hacker_cluster_2.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

In [52]:
hacker_cluster_3.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+

# Answer: 2 hackers did the job